# Final Report

Importing Lib.

In [1]:
import numpy as np # Library, vectorize data

import pandas as pd # library, data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library, JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # get geo coordinates per area
!pip install geocoder
import geocoder

import requests # library, requests
from bs4 import BeautifulSoup # library, parse HTML and XML files
from pandas.io.json import json_normalize # json file into pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Scrap Data

In [2]:
data = requests.get("https://pl.wikipedia.org/wiki/Kategoria:Dzielnice_i_osiedla_Olsztyna").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
o_df = pd.DataFrame({"Neighborhood": neighborhoodList})
o_df.head()

,Neighborhood
0,Podział administracyjny Olsztyna
1,Brzeziny (Olsztyn)
2,Dajtki
3,Gutkowo (Olsztyn)
4,Jakubowo (Olsztyn)


Now we need to remove 'Podzial administracyjnu Olsztyna', as it is essencially name of the category and not area name.

In [3]:
o_df.drop(o_df.index[0], inplace=True)
o_df.head()

,Neighborhood
1,Brzeziny (Olsztyn)
2,Dajtki
3,Gutkowo (Olsztyn)
4,Jakubowo (Olsztyn)
5,Jaroty


In [4]:
o_df.shape

(40, 1)

# Get Geo-Co

In [5]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Olsztyn, Poland'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [6]:
coords = [ get_latlng(neighborhood) for neighborhood in o_df["Neighborhood"].tolist() ]
coords

[[53.77900000000005, 20.480230000000063],
 [53.77900000000005, 20.480230000000063],
 [53.80905150000002, 20.38818150000001],
 [53.77778997778305, 20.478060012262393],
 [53.728059210086286, 20.476762171805678],
 [53.77900000000005, 20.480230000000063],
 [53.728059210086286, 20.476762171805678],
 [53.77900000000005, 20.480230000000063],
 [53.77900000000005, 20.480230000000063],
 [53.76923099999999, 20.481945750000012],
 [53.77900000000005, 20.480230000000063],
 [53.78633488675882, 20.378047545349443],
 [53.77900000000005, 20.480230000000063],
 [50.75657861578568, 19.261190212657297],
 [53.77900000000005, 20.480230000000063],
 [53.77694425061457, 20.46702021538436],
 [53.781905953390535, 20.495905077682433],
 [53.77177000000006, 20.504820000000052],
 [53.75874470708431, 20.50498828378802],
 [53.77900000000005, 20.480230000000063],
 [53.785600051070496, 20.45815999220979],
 [53.7497917521916, 20.511897558738596],
 [53.81160299999999, 20.479828499999996],
 [53.76667000000003, 20.53333000000

In [7]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
o_df['Latitude'] = df_coords['Latitude']
o_df['Longitude'] = df_coords['Longitude']
print(o_df.shape)

(40, 3)


In [8]:
o_df.to_csv("o_df.csv", index=False)

In [9]:
o_df.head(50)

,Neighborhood,Latitude,Longitude
1,Brzeziny (Olsztyn),53.779000,20.480230
2,Dajtki,53.809052,20.388182
3,Gutkowo (Olsztyn),53.777790,20.478060
4,Jakubowo (Olsztyn),53.728059,20.476762
5,Jaroty,53.779000,20.480230
6,Karolin (Olsztyn),53.728059,20.476762
7,Kolonia Jaroty,53.779000,20.480230
8,Kortowo,53.779000,20.480230
9,Kortowo II,53.769231,20.481946
10,Kościuszki (Olsztyn),53.779000,20.480230


We have to drop 'Zielona Gorka' as we could not find its coordinates:

In [10]:
o_df.drop(o_df.index[39], inplace=True)
o_df.head(50)

,Neighborhood,Latitude,Longitude
1,Brzeziny (Olsztyn),53.779000,20.480230
2,Dajtki,53.809052,20.388182
3,Gutkowo (Olsztyn),53.777790,20.478060
4,Jakubowo (Olsztyn),53.728059,20.476762
5,Jaroty,53.779000,20.480230
6,Karolin (Olsztyn),53.728059,20.476762
7,Kolonia Jaroty,53.779000,20.480230
8,Kortowo,53.779000,20.480230
9,Kortowo II,53.769231,20.481946
10,Kościuszki (Olsztyn),53.779000,20.480230


# Create a map with neighborhoods positioned on top

In [11]:
address = 'Olsztyn, Poland'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('You can find Olsztyn, Poland at {} latitude, {} longitude, the Earth, the Solar System, Orion Arm, the Milky Way, the Local Group, the Virgo Supercluster, the Universe.'.format(latitude, longitude))

You can find Olsztyn, Poland at 53.7767239 latitude, 20.4777812922103 longitude, the Earth, the Solar System, Orion Arm, the Milky Way, the Local Group, the Virgo Supercluster, the Universe.


In [12]:
map_o = folium.Map(location=[latitude, longitude], zoom_start=12)

In [13]:
for lat, lng, neighborhood in zip(o_df['Latitude'], o_df['Longitude'], o_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3139').add_to(map_o)
map_o

In [14]:
map_o.save('map_o.html')

# Use ForsquareAPI to explore the area

In [15]:
CLIENT_ID = 'O0WRXGSLTYFQDZSMYOBNOJVCLQMZO2MSSC3ZGT2SYEZDX5ZY'
CLIENT_SECRET = 'RMJLNENLVVACHDXTZBGDQ3XSUFPXFZNNSEWD21UW4LBTW3IX' 
VERSION = '20190725'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O0WRXGSLTYFQDZSMYOBNOJVCLQMZO2MSSC3ZGT2SYEZDX5ZY
CLIENT_SECRET:RMJLNENLVVACHDXTZBGDQ3XSUFPXFZNNSEWD21UW4LBTW3IX


Let's show top 100 venues in the radious of 2000 meters:

In [32]:
radius = 2000
LIMIT = 100
venues = []
for lat, long, neighborhood in zip(o_df['Latitude'], o_df['Longitude'], o_df['Neighborhood']):

    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [33]:
# create Data Frame out of venue list
venues_df = pd.DataFrame(venues)
# give column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1257, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Brzeziny (Olsztyn),53.779,20.48023,Browar Warmia,53.778707,20.474518,Brewery
1,Brzeziny (Olsztyn),53.779,20.48023,Jam-Jam,53.775316,20.475334,Fast Food Restaurant
2,Brzeziny (Olsztyn),53.779,20.48023,Starówka,53.776521,20.475672,Plaza
3,Brzeziny (Olsztyn),53.779,20.48023,Drewno I Ogień,53.775597,20.475649,Pizza Place
4,Brzeziny (Olsztyn),53.779,20.48023,Irish Pub Carpenter Inn,53.777324,20.476621,Pub


Lets see how many venues were returned for each area:

In [34]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Brzeziny (Olsztyn),51,51,51,51,51,51
Dajtki,4,4,4,4,4,4
Gutkowo (Olsztyn),53,53,53,53,53,53
Jakubowo (Olsztyn),4,4,4,4,4,4
Jaroty,51,51,51,51,51,51
Karolin (Olsztyn),4,4,4,4,4,4
Kolonia Jaroty,51,51,51,51,51,51
Kortowo,51,51,51,51,51,51
Kortowo II,53,53,53,53,53,53


# Analyse each area

In [36]:
o_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add area column back to dataframe
o_onehot['Neighborhoods'] = venues_df['Neighborhood'] 
# move area column to the first column
fixed_columns = [o_onehot.columns[-1]] + list(o_onehot.columns[:-1])
o_onehot = o_onehot[fixed_columns]

print(o_onehot.shape)
o_onehot.head()

(1257, 61)


,Neighborhoods,American Restaurant,Beach,Bowling Alley,Brewery,Bus Station,Café,Campground,Castle,Clothing Store,Construction & Landscaping,Diner,Dumpling Restaurant,Electronics Store,Factory,Fast Food Restaurant,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gun Range,Gym,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,Hotel,Italian Restaurant,Juice Bar,Lake,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motel,Multiplex,Music Venue,Nightclub,Park,Performing Arts Venue,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Recreation Center,Restaurant,Sandwich Place,Shipping Store,Shop & Service,Shopping Mall,Ski Area,Soccer Field,Steakhouse,Supermarket,Train Station,Vegetarian / Vegan Restaurant,Warehouse Store,Zoo
0,Brzeziny (Olsztyn),0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Brzeziny (Olsztyn),0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Brzeziny (Olsztyn),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Brzeziny (Olsztyn),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Brzeziny (Olsztyn),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
o_grouped = o_onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(o_grouped.shape)
o_grouped

(38, 61)


,Neighborhoods,American Restaurant,Beach,Bowling Alley,Brewery,Bus Station,Café,Campground,Castle,Clothing Store,Construction & Landscaping,Diner,Dumpling Restaurant,Electronics Store,Factory,Fast Food Restaurant,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gun Range,Gym,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,Hotel,Italian Restaurant,Juice Bar,Lake,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motel,Multiplex,Music Venue,Nightclub,Park,Performing Arts Venue,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Recreation Center,Restaurant,Sandwich Place,Shipping Store,Shop & Service,Shopping Mall,Ski Area,Soccer Field,Steakhouse,Supermarket,Train Station,Vegetarian / Vegan Restaurant,Warehouse Store,Zoo
0,Brzeziny (Olsztyn),0.019608,0.000000,0.0,0.019608,0.019608,0.058824,0.0,0.019608,0.000000,0.000000,0.019608,0.019608,0.000000,0.000000,0.098039,0.0,0.019608,0.000000,0.000000,0.0,0.019608,0.000000,0.000000,0.000000,0.039216,0.137255,0.019608,0.000000,0.000000,0.019608,0.019608,0.019608,0.000000,0.000000,0.0,0.019608,0.019608,0.000000,0.019608,0.0,0.019608,0.019608,0.019608,0.039216,0.019608,0.078431,0.000000,0.039216,0.019608,0.000000,0.000000,0.039216,0.000000,0.019608,0.000000,0.019608,0.019608,0.019608,0.000000,0.000000
1,Dajtki,0.000000,0.250000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Gutkowo (Olsztyn),0.018868,0.018868,0.0,0.018868,0.000000,0.075472,0.0,0.018868,0.000000,0.000000,0.018868,0.018868,0.000000,0.000000,0.094340,0.0,0.018868,0.000000,0.000000,0.0,0.037736,0.000000,0.018868,0.000000,0.037736,0.094340,0.018868,0.000000,0.000000,0.018868,0.018868,0.018868,0.000000,0.000000,0.0,0.018868,0.018868,0.000000,0.018868,0.0,0.018868,0.018868,0.018868,0.037736,0.018868,0.075472,0.018868,0.056604,0.018868,0.000000,0.000000,0.037736,0.000000,0.000000,0.000000,0.018868,0.018868,0.018868,0.000000,0.000000
3,Jakubowo (Olsztyn),0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000
4,Jaroty,0.019608,0.000000,0.0,0.019608,0.019608,0.058824,0.0,0.019608,0.000000,0.000000,0.019608,0.019608,0.000000,0.000000,0.098039,0.0,0.019608,0.000000,0.000000,0.0,0.019608,0.000000,0.000000,0.000000,0.039216,0.137255,0.019608,0.000000,0.000000,0.019608,0.019608,0.019608,0.000000,0.000000,0.0,0.019608,0.019608,0.000000,0.019608,0.0,0.019608,0.019608,0.019608,0.039216,0.019608,0.078431,0.000000,0.039216,0.019608,0.000000,0.000000,0.039216,0.000000,0.019608,0.000000,0.019608,0.019608,0.019608,0.000000,0.000000
5,Karolin (Olsztyn),0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000
6,Kolonia Jaroty,0.019608,0.000000,0.0,0.019608,0.01

Creating DataFrame

Due to the fact that we dont have a lot of venue data for Olsztyn, I changed the scope of the task and focused only on sport - related activities, as they would be important for me while making a purchase decision

In [47]:
o_sport = o_grouped[["Neighborhoods","Beach", "Gym", "Gym Pool", "Harbor / Marina", "Lake", "Pool", "Ski Area", "Soccer Field"]]
o_sport.head()

,Neighborhoods,Beach,Gym,Gym Pool,Harbor / Marina,Lake,Pool,Ski Area,Soccer Field
0,Brzeziny (Olsztyn),0.000000,0.019608,0.0,0.000000,0.00,0.019608,0.0,0.019608
1,Dajtki,0.250000,0.000000,0.0,0.000000,0.25,0.000000,0.0,0.000000
2,Gutkowo (Olsztyn),0.018868,0.037736,0.0,0.018868,0.00,0.018868,0.0,0.000000
3,Jakubowo (Olsztyn),0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.0,0.000000
4,Jaroty,0.000000,0.019608,0.0,0.000000,0.00,0.019608,0.0,0.019608


# Cluster Neighborhoods

In [48]:
# set number of clusters
kclusters = 3
o_clustering = o_sport.drop(["Neighborhoods"], 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(o_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 2, 0, 2, 0, 0, 0, 0], dtype=int32)

In [49]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
o_merged = o_sport.copy()
# add clustering labels
o_merged["Cluster Labels"] = kmeans.labels_

In [50]:
o_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
o_merged.head()

,Neighborhood,Beach,Gym,Gym Pool,Harbor / Marina,Lake,Pool,Ski Area,Soccer Field,Cluster Labels
0,Brzeziny (Olsztyn),0.000000,0.019608,0.0,0.000000,0.00,0.019608,0.0,0.019608,0
1,Dajtki,0.250000,0.000000,0.0,0.000000,0.25,0.000000,0.0,0.000000,2
2,Gutkowo (Olsztyn),0.018868,0.037736,0.0,0.018868,0.00,0.018868,0.0,0.000000,0
3,Jakubowo (Olsztyn),0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.0,0.000000,2
4,Jaroty,0.000000,0.019608,0.0,0.000000,0.00,0.019608,0.0,0.019608,0


In [51]:
o_merged = o_merged.join(o_df.set_index("Neighborhood"), on="Neighborhood")
print(o_merged.shape)
o_merged.head()

(38, 12)


,Neighborhood,Beach,Gym,Gym Pool,Harbor / Marina,Lake,Pool,Ski Area,Soccer Field,Cluster Labels,Latitude,Longitude
0,Brzeziny (Olsztyn),0.000000,0.019608,0.0,0.000000,0.00,0.019608,0.0,0.019608,0,53.779000,20.480230
1,Dajtki,0.250000,0.000000,0.0,0.000000,0.25,0.000000,0.0,0.000000,2,53.809052,20.388182
2,Gutkowo (Olsztyn),0.018868,0.037736,0.0,0.018868,0.00,0.018868,0.0,0.000000,0,53.777790,20.478060
3,Jakubowo (Olsztyn),0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.0,0.000000,2,53.728059,20.476762
4,Jaroty,0.000000,0.019608,0.0,0.000000,0.00,0.019608,0.0,0.019608,0,53.779000,20.480230


In [46]:
# sort the results by Cluster Labels
print(o_merged.shape)
o_merged.sort_values(["Cluster Labels"], inplace=True)
o_merged

(38, 12)


,Neighborhood,Beach,Gym,Gym Pool,Harbor / Marina,Lake,Pool,Ski Area,Soccer Field,Cluster Labels,Latitude,Longitude
1,Dajtki,0.250000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0,53.809052,20.388182
34,Słoneczny Stok (Olsztyn),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,1,53.821400,20.508400
31,Stare Miasto (Olsztyn),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1,53.815228,20.545305
3,Jakubowo (Olsztyn),0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,2,53.728059,20.476762
5,Karolin (Olsztyn),0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,2,53.728059,20.476762
29,Skarbówka Poszmanówka,0.000000,0.000000,0.166667,0.000000,0.166667,0.000000,0.000000,0.000000,2,53.749311,20.447712
21,Osiedle Wojska Polskiego (Olsztyn),0.000000,0.200000,0.000000,0.000000,0.200000,0.200000,0.000000,0.200000,3,53.766670,20.533330
0,Brzeziny (Olsztyn),0.000000,0.019608,0.000000,0.000000,0.000000,0.019608,0.000000,0.019608,4,53.779000,20.480230
22,Pieczewo,0.021277,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4,53.769350,20.474920
23,Podgrodzie (Olsztyn),0.000000,0.019608,0.000000,0.000000,0.000000,0.019608,0.000000,0.019608,4,53.779000,20.480230


Lets visualise the clusters on a map:

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(o_merged['Latitude'], o_merged['Longitude'], o_merged['Neighborhood'], o_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

In [50]:
map_clusters.save('map_clusters.html')

# Examine Clusters

Cluster 0

In [53]:
o_merged.loc[o_merged['Cluster Labels'] == 0]

,Neighborhood,Beach,Gym,Gym Pool,Harbor / Marina,Lake,Pool,Ski Area,Soccer Field,Cluster Labels,Latitude,Longitude
0,Brzeziny (Olsztyn),0.000000,0.019608,0.0,0.000000,0.000000,0.019608,0.0,0.019608,0,53.779000,20.480230
2,Gutkowo (Olsztyn),0.018868,0.037736,0.0,0.018868,0.000000,0.018868,0.0,0.000000,0,53.777790,20.478060
4,Jaroty,0.000000,0.019608,0.0,0.000000,0.000000,0.019608,0.0,0.019608,0,53.779000,20.480230
6,Kolonia Jaroty,0.000000,0.019608,0.0,0.000000,0.000000,0.019608,0.0,0.019608,0,53.779000,20.480230
7,Kortowo,0.000000,0.019608,0.0,0.000000,0.000000,0.019608,0.0,0.019608,0,53.779000,20.480230
8,Kortowo II,0.000000,0.018868,0.0,0.000000,0.000000,0.018868,0.0,0.018868,0,53.769231,20.481946
9,Kościuszki (Olsztyn),0.000000,0.019608,0.0,0.000000,0.000000,0.019608,0.0,0.019608,0,53.779000,20.480230
10,Likusy (Olsztyn),0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0,53.786335,20.378048
11,Nagórki (Olsztyn),0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0,50.756579,19.261190
12,Niedźwiedź (Olsztyn),0.000000,0.019608,0.0,0.000000,0.000000,0.019608,0.0,0.019608,0,53.779000,20.480230


Cluster 1

In [54]:
o_merged.loc[o_merged['Cluster Labels'] == 1]

,Neighborhood,Beach,Gym,Gym Pool,Harbor / Marina,Lake,Pool,Ski Area,Soccer Field,Cluster Labels,Latitude,Longitude
31,Stare Miasto (Olsztyn),0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,1,53.815228,20.545305
34,Słoneczny Stok (Olsztyn),0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,1,53.821400,20.508400


Cluster 2

In [55]:
o_merged.loc[o_merged['Cluster Labels'] == 2]

,Neighborhood,Beach,Gym,Gym Pool,Harbor / Marina,Lake,Pool,Ski Area,Soccer Field,Cluster Labels,Latitude,Longitude
1,Dajtki,0.25,0.0,0.000000,0.0,0.250000,0.0,0.0,0.0,2,53.809052,20.388182
3,Jakubowo (Olsztyn),0.00,0.0,0.000000,0.0,0.250000,0.0,0.0,0.0,2,53.728059,20.476762
5,Karolin (Olsztyn),0.00,0.0,0.000000,0.0,0.250000,0.0,0.0,0.0,2,53.728059,20.476762
21,Osiedle Wojska Polskiego (Olsztyn),0.00,0.2,0.000000,0.0,0.200000,0.2,0.0,0.2,2,53.766670,20.533330
29,Skarbówka Poszmanówka,0.00,0.0,0.166667,0.0,0.166667,0.0,0.0,0.0,2,53.749311,20.447712


# Observations

I much say, I was disappointed with the quantity of venues available for Olsztyn and the number of categories available. I should have checked it before starting this project, but it’s certainly a lesson I will remember. I had to choose at some point categories that were in a quantity significant enough to make a difference and relevant to the project. I personally like sports; hence I selected those categories with the highest proxy to sports. It seems that most venues are in Cluster 0. This would indicate where one would have to look to buy a property.